In [ ]:
!pip install lac
!pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
  Created wheel for lac: filename=LAC-2.1.2-py2.py3-none-any.whl size=64814699 sha256=ddff62350c52b79433503b424314b428bc8feae3ee4107d99572ebd04d221dd5
  Stored in directory: /root/.cache/pip/wheels/5b/a0/02/8fac032f2babbef354a8911bb7ad903f9cfc849458fdeab295
Successfully built lac
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 6.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# # 2. Use gdown tool to download from Google Drive link
!pip install gdown
import gdown
url = 'https://drive.google.com/u/0/uc?id=1VAzteetSSc9WOCne_u6-5oFt_6rIMR5E&export=download'
output = '/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/news_data.xlsx'
gdown.download(url, output, quiet=False)

Mounted at /content/drive


Downloading...
From: https://drive.google.com/u/0/uc?id=1VAzteetSSc9WOCne_u6-5oFt_6rIMR5E&export=download
To: /content/drive/MyDrive/DSAA5002PROJECT/Task1_data/news_data.xlsx
100%|██████████| 557M/557M [00:18<00:00, 30.4MB/s]


'/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/news_data.xlsx'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# loading Data
news_data = pd.read_excel('/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/news_data.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
news_data.head()

,NewsID,Title,NewsContent,NewsSource
0,1,建设银行原董事长张恩照一审被判15年,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...,中国证券报
1,2,农行信用卡中心搬到上海滩,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上...,人民日报
2,3,外运发展：价值型蓝筹股补涨要求强烈,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...,杭州新希望
3,4,胜利股份：稳步走强形成标准上升通道,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿...,源达投资
4,5,[港股快讯]恒指收市报18960点 成交467亿港元,全景网11月30日讯 外围股市造好，带动港股今早造好，恒指高开后反覆上升，最高升252点...,全景网


In [ ]:
news_data.shape

(1037035, 4)

In [ ]:
import re
import pandas as pd
import json
from rapidfuzz import process, fuzz

# Preliminary
json_path = '/content/drive/MyDrive/DSAA5002PROJECT/dsaa5002project/dsaa5002_project/A_share_list.json'
news_data['Title'] = news_data['Title'].astype(str)
news_data['NewsContent'] = news_data['NewsContent'].astype(str)
df = news_data[:240000]

# Read json file
with open(json_path, 'r', encoding='utf-8') as f:
    stock_list = json.load(f)

abbreviation_to_name = {
    'ST亚星': '亚星化学',
    'SST丰华': '丰华股份',
    '中信': '中信证券',
    '中金': '中金公司',
    '中石油': '中国石油',
    '中石化': '中国石化',
    '中钢': '中钢国际',
    '招行': '招商银行',
    '工行': '工商银行',
    '交行': '交通银行',
    '中行': '中国银行',
    '建行': '建设银行',
    '国寿': '中国人寿',
    '平保': '中国太保',
    '南航': '南方航空',
    '海航': '海南航空',
    '万科': '万科A',
    '茅台': '贵州茅台',
    '海尔': '青岛海尔'
}

# Prepare mapping of stock names, full names and codes
stock_info = {}
for stock in stock_list:
    name = stock['name']
    fullname = re.sub('股份有限公司$', '', stock['fullname'])  # Remove the 'company limited by shares' suffix
    code = re.sub('\D', '', stock['code'])  # retain figure
    # Mapping full names and codes to names
    stock_info[name] = name
    stock_info[fullname] = name
    stock_info[code] = name

# Importing newly defined variant mappings
for abbreviation, name in abbreviation_to_name.items():
    stock_info[abbreviation] = name

from LAC import LAC

# Load LAC model
lac = LAC(mode='lac')

def extract_org_names(text):
    # Using LAC to process text
    lac_result = lac.run(text)
    words = lac_result[0]  # Segmentation results
    tags = lac_result[1]  # Lexical and entity labeling results

    # Extract all entities of type ORGs
    org_entities = [word for word, tag in zip(words, tags) if tag == 'ORG']
    # Remove duplicate ORG entities
    unique_org_entities = list(set(org_entities))
    return unique_org_entities

def find_stock_names(org_names, stock_names, threshold=90):
    matched_names = set()
    for org_name in org_names:
        # Specific Matching Rules
        if org_name in ['中国人民银行', '中国央行', '央行', '中金所', '中国金融期货交易所', '公司', '京东', '美团']:
            continue
        else:
            # Fuzzy matching with rapidfuzz
            best_match = process.extractOne(org_name, stock_names, scorer=fuzz.QRatio, score_cutoff=threshold)
            if best_match:
                # Determine if a character is a subset
                if set(best_match[0]).issubset(set(org_name)) or set(org_name).issubset(set(best_match[0])):
                    matched_names.add(best_match[0])
    return matched_names

with open(json_path, 'r', encoding='utf-8') as f:
    stock_list = json.load(f)
stock_names = [stock['name'] for stock in stock_list]

def extract_stock_names(text, stock_info, stock_names):
    # Check if text is a string
    if not isinstance(text, str):
        return ''

    # Extract organization name
    org_names = extract_org_names(text)

    # Find stock names with find_stock_names
    fuzzy_matched_names = find_stock_names(org_names, stock_names)

    text_no_stars = text.replace('*', '')

    direct_matched_names = set()
    for variant, name in stock_info.items():
        if variant in text_no_stars:
            direct_matched_names.add(name)

    matched_names = direct_matched_names.union(fuzzy_matched_names)

    return ','.join(matched_names)

# Apply functions and create new features
df['Explicit_Company'] = df.apply(lambda row: extract_stock_names(row['Title'], stock_info, stock_names) + ',' + extract_stock_names(row['NewsContent'], stock_info, stock_names), axis=1)
df['Explicit_Company'] = df['Explicit_Company'].str.strip(',')

def remove_duplicate_names(names):
    unique_names = set(names.split(','))
    return ','.join(unique_names)

# Apply the function to the 'Explicit_Company' column to remove duplicates
df['Explicit_Company'] = df['Explicit_Company'].apply(remove_duplicate_names)

# Now you can proceed with filtering the DataFrame
filtered_df = df[df['Explicit_Company'].apply(lambda x: len(x) > 0)]

# Output the result
print(len(filtered_df))


<ipython-input-19-a522873b24c7>:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Explicit_Company'] = df.apply(lambda row: extract_stock_names(row['Title'], stock_info, stock_names) + ',' + extract_stock_names(row['NewsContent'], stock_info, stock_names), axis=1)
<ipython-input-19-a522873b24c7>:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Explicit_Company'] = df['Explicit_Company'].str.strip(',')  # 移除多余的逗号


108662


<ipython-input-19-a522873b24c7>:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Explicit_Company'] = df['Explicit_Company'].apply(remove_duplicate_names)


In [ ]:
filtered_df

,NewsID,Title,NewsContent,NewsSource,Explicit_Company
0,1,建设银行原董事长张恩照一审被判15年,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...,中国证券报,建设银行
1,2,农行信用卡中心搬到上海滩,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上...,人民日报,农业银行
2,3,外运发展：价值型蓝筹股补涨要求强烈,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...,杭州新希望,"中国国航,外运发展"
3,4,胜利股份：稳步走强形成标准上升通道,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿...,源达投资,胜利股份
4,5,[港股快讯]恒指收市报18960点 成交467亿港元,全景网11月30日讯 外围股市造好，带动港股今早造好，恒指高开后反覆上升，最高升252点...,全景网,"中国银行,建设银行,工商银行,中国太保,交通银行,中国人寿,招商银行"
...,...,...,...,...,...
239979,239980,弘扬风险管理文化 中金所颁发功能发挥奖,证券时报记者 游石 为发挥市场功能，弘扬风险管理文化，中国金融期货交易所（以下简称中...,证券时报,中金公司
239982,239983,“喝酒吃药”两券商理财产品去年很滋补,证券时报记者 杜妍 近日，券商集合理财2010年四季度资产管理报告陆续公布。去年收益...,证券时报,"国中水务,敦煌种业,亚厦股份,时代新材,海印股份"
239987,239988,“破发潮”下券商思变借伙伴分摊“被股东”风险,证券时报记者 张伟霖 投资者用脚投票所导致的新股频频破发，使得券商承销公开增发项目结...,证券时报,"长安汽车,国电电力"
239990,239991,房地产商：楼市或向买方市场转变,证券时报记者 向南 针对昨日国务院正式批复同意房产税在全国部分城市进行试点一事，证券...,证券时报,"招商地产,万科A"


In [ ]:
Be_filtered_df = df[df['Explicit_Company'].apply(lambda x: len(x) == 0)]
Be_filtered_df.to_excel('/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/Flitered/Be_filtered-part1.xlsx' ,encoding='utf-8')
filtered_df.to_excel('/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/Flitered/Task1_part1-1.xlsx' ,encoding='utf-8')

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
